In [37]:
%run file_paths_updt Copy


StatementMeta(, d8615804-30dd-4c9e-a4b6-636f451cec6c, 39, Finished, Available, Finished)

In [2]:
df = spark.read.format('delta').load(Silver+'//'+'Silver_table')

StatementMeta(, d8615804-30dd-4c9e-a4b6-636f451cec6c, 4, Finished, Available, Finished)

In [3]:
display(df)

StatementMeta(, d8615804-30dd-4c9e-a4b6-636f451cec6c, 5, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 952a0474-6c92-484b-8982-53579d6da1a8)

In [4]:
display(df.select('gender').distinct())

StatementMeta(, d8615804-30dd-4c9e-a4b6-636f451cec6c, 6, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 1888900d-7814-424e-b4d4-b7cfe4ceab0b)

In [5]:
dfnew = df.select('employee_id','first_name','last_name','birthdate','gender','race','hire_date','termdate')

StatementMeta(, d8615804-30dd-4c9e-a4b6-636f451cec6c, 7, Finished, Available, Finished)

In [6]:
display(dfnew)

StatementMeta(, d8615804-30dd-4c9e-a4b6-636f451cec6c, 8, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 92755a47-665e-48ea-97ad-71bb1eda797b)

In [7]:
from pyspark.sql.functions import *

StatementMeta(, d8615804-30dd-4c9e-a4b6-636f451cec6c, 9, Finished, Available, Finished)

In [8]:
dim_emp = dfnew.withColumn(
    "gender",
    when(col("gender") == "M", "Male")
    .when(col("gender") == "FM", "Female")
    .otherwise('Other') )\
    .withColumn('Full_Name',concat(col('first_name'),lit(" "),col('last_name')))\
    .withColumnRenamed('termdate','Termination_date')


StatementMeta(, d8615804-30dd-4c9e-a4b6-636f451cec6c, 10, Finished, Available, Finished)

In [9]:
display(dim_emp)

StatementMeta(, d8615804-30dd-4c9e-a4b6-636f451cec6c, 11, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, a0d36e52-3187-4012-a0e6-d38eb6e5c499)

In [10]:
df2 = df.select('location_state','location_city').distinct()

StatementMeta(, d8615804-30dd-4c9e-a4b6-636f451cec6c, 12, Finished, Available, Finished)

In [11]:
from pyspark.sql.window import Window
from pyspark.sql.types import *

StatementMeta(, d8615804-30dd-4c9e-a4b6-636f451cec6c, 13, Finished, Available, Finished)

In [12]:
dim_loc = df2.withColumn('location_id',row_number().over(Window.orderBy('location_state')))\
             .withColumn('location',struct(col('location_city').alias('city'),col('location_state').alias('State')))

StatementMeta(, d8615804-30dd-4c9e-a4b6-636f451cec6c, 14, Finished, Available, Finished)

In [13]:
display(dim_loc)

StatementMeta(, d8615804-30dd-4c9e-a4b6-636f451cec6c, 15, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, ddd770be-0971-4a9a-a908-65bf8b26def8)

In [14]:
df3 = df.select('department').distinct()

StatementMeta(, d8615804-30dd-4c9e-a4b6-636f451cec6c, 16, Finished, Available, Finished)

In [15]:
dim_dept = df3.withColumn('Department_id',row_number().over(Window.orderBy('department')))

StatementMeta(, d8615804-30dd-4c9e-a4b6-636f451cec6c, 17, Finished, Available, Finished)

In [16]:
display(dim_dept)

StatementMeta(, d8615804-30dd-4c9e-a4b6-636f451cec6c, 18, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 97909621-dc6f-4c22-b5c5-77035d93c8dc)

In [17]:
df4 = df.join(dim_loc,on=['location_city','location_state'],how='inner')

StatementMeta(, d8615804-30dd-4c9e-a4b6-636f451cec6c, 19, Finished, Available, Finished)

In [18]:
display(df4)

StatementMeta(, d8615804-30dd-4c9e-a4b6-636f451cec6c, 20, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 042910c6-8f89-40d7-b81a-19ca27fa4f89)

In [19]:
df5 = df4.join(dim_dept,on='department',how='inner')

StatementMeta(, d8615804-30dd-4c9e-a4b6-636f451cec6c, 21, Finished, Available, Finished)

In [20]:
df5.count()

StatementMeta(, d8615804-30dd-4c9e-a4b6-636f451cec6c, 22, Finished, Available, Finished)

22214

In [21]:
display(df5)

StatementMeta(, d8615804-30dd-4c9e-a4b6-636f451cec6c, 23, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, efc054db-d563-4c80-93d7-7da719bc746c)

In [22]:
df6 = df5.withColumn('total_emp',count('employee_id').over(Window.partitionBy('Department_id')))\
         .withColumn('age', floor(months_between(current_date(), col("birthdate")) / 12))\
         .withColumn('avg_age_by_dept',floor(avg('age').over(Window.partitionBy('Department_id'))))\
         .withColumn(
            "gender",
            when(col("gender") == "M", "Male")
            .when(col("gender") == "FM", "Female")
            .otherwise('Other') )


StatementMeta(, d8615804-30dd-4c9e-a4b6-636f451cec6c, 24, Finished, Available, Finished)

In [23]:
display(df6)

StatementMeta(, d8615804-30dd-4c9e-a4b6-636f451cec6c, 25, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 09ab5aa5-05e4-4118-a081-60bff213cc2c)

In [24]:
eng = df6.select('Department_id','age','avg_age_by_dept','total_emp').filter(col('Department_id')==2)

StatementMeta(, d8615804-30dd-4c9e-a4b6-636f451cec6c, 26, Finished, Available, Finished)

In [25]:
from pyspark.sql.functions import round, col, sum, concat_ws, lit, collect_list
from pyspark.sql.window import Window

# Step 1: Count genders per department
gender_counts = df6.groupBy("department", "gender").count()

# Step 2: Calculate percentage per gender per department
gender_percentages = gender_counts.withColumn(
    "percentage",
    round((col("count") / sum("count").over(Window.partitionBy("department"))) * 100, 2)
)

# Step 3: Format as "gender:xx%"
gender_percentages = gender_percentages.withColumn(
    "gender_percentage",
    concat_ws("", col("gender"), lit(":"), col("percentage").cast("string"), lit("%"))
)

# Step 4: Aggregate all gender_percentage strings into a single string per department
df_7 = gender_percentages.groupBy("department").agg(
    concat_ws(", ", collect_list("gender_percentage")).alias("gender_distribution")
)

display(df_7)


StatementMeta(, d8615804-30dd-4c9e-a4b6-636f451cec6c, 27, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, c1b601c5-498d-4a91-8395-9e98bfda87a7)

In [26]:
df_8 = df6.join(df_7,on=['department'],how='inner')
display(df_8)

StatementMeta(, d8615804-30dd-4c9e-a4b6-636f451cec6c, 28, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, c63185ae-e1f3-496d-8c5a-01db8fdbea6f)

In [27]:
df_8.count()

StatementMeta(, d8615804-30dd-4c9e-a4b6-636f451cec6c, 29, Finished, Available, Finished)

22214

In [28]:
df_with_in_active = df_8.withColumn(
    'inactive_emp',
    sum(when(col('termdate').isNotNull(), 1).otherwise(0)).over(Window.partitionBy('Department_id'))
)

display(df_with_in_active)
df_with_in_active.count()

StatementMeta(, d8615804-30dd-4c9e-a4b6-636f451cec6c, 30, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, e873c7cf-6eea-4b53-9686-2c1a570e024e)

22214

In [29]:
hr_fact = df_with_in_active.withColumn('turnover_rate', round((col('inactive_emp') / col('total_emp')) * 100, 2))\
                           .select('employee_id','location_id','department_id','total_emp','avg_age_by_dept','gender_distribution','turnover_rate')

StatementMeta(, d8615804-30dd-4c9e-a4b6-636f451cec6c, 31, Finished, Available, Finished)

In [30]:
display(hr_fact)

StatementMeta(, d8615804-30dd-4c9e-a4b6-636f451cec6c, 32, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, f77c8bb0-4965-4bc4-a3e9-a7c521b77374)

In [38]:
print(Gold)

StatementMeta(, d8615804-30dd-4c9e-a4b6-636f451cec6c, 40, Finished, Available, Finished)

abfss://FabricTrainingWorkspace@onelake.dfs.fabric.microsoft.com/Rughwed_Gold.Lakehouse/Tables


In [39]:
dim_emp.write.format('delta').save(Gold+'//'+'emp_dim')

StatementMeta(, d8615804-30dd-4c9e-a4b6-636f451cec6c, 41, Finished, Available, Finished)

In [40]:
dim_loc.write.format('delta').save(Gold+'//'+'emp_loc')
dim_dept.write.format('delta').save(Gold+'//'+'emp_dept')
hr_fact.write.format('delta').save(Gold+'//'+'Hr_fact')

StatementMeta(, d8615804-30dd-4c9e-a4b6-636f451cec6c, 42, Finished, Available, Finished)